# Introduction

Delaney had implemented a method to resolve transcript compatibility counts from kallisto into transcript level quantifications with 10x / droplet data.

I wonder how it performs? None of the other 10x methods produce transcript level data.

I guess I'll try comparing the run I did to our C1 data.

I had randomly picked 10x-3 (e11.0)

In [22]:
import scanpy
import numpy
import pandas
import sys
from pathlib import Path

In [25]:
LRS = str(Path("~/proj/long-rna-seq-condor").expanduser())
if LRS not in sys.path:
    sys.path.append(LRS)
from woldrnaseq import models

In [2]:
kallisto_em_root = Path('~/proj/encode-202006-jamboree-detrout-rna-sc-pipeline/10x_paper/kallisto_em/').expanduser()

In [19]:
def scanpy_load_kallisto_tcc_mtx(analysis_dir, filter_file=None):
    analysis_dir = Path(analysis_dir)
    quant_dir = analysis_dir / "quant_tcc"
    kallisto = scanpy.read_mtx(quant_dir / "matrix.abundance.mtx")

    kallisto_vars = pandas.read_csv(
        quant_dir / "transcripts.txt", header=None
    ).values.T
    kallisto_obs = pandas.read_csv(
        analysis_dir / "quant_umi" / "output.barcodes.txt", header=None
    ).values.T
    kallisto.obs_names = kallisto_obs[0]
    kallisto.var_names = kallisto_vars[0]

    if filter_file is not None:
        barcodes = load_barcode_list(filter_file)
        kallisto = kallisto[barcodes].copy()

    kallisto.obs["counts"] = kallisto.X.sum(axis=1)
    kallisto.obs["ngenes"] = numpy.array((kallisto.X > 0).sum(axis=1))

    return kallisto

In [20]:
tenx3_e12 = scanpy_load_kallisto_tcc_mtx(kallisto_em_root / "10x-3")

In [21]:
tenx3_e12.shape

(655514, 168207)

In [30]:
c1_m21_root = Path('~/proj/C1_mouse_limb_combined/all_analysis_M21/').expanduser()
c1_m21_library_tsv = c1_m21_root / 'libraries-passing.tsv'
c1_m21_experiments_tsv = c1_m21_root / 'experiments-by-run-passing.tsv'

In [31]:
c1_m21_library = models.load_library_tables([c1_m21_library_tsv])
c1_m21_experiment = models.load_experiments([c1_m21_experiments_tsv])

In [35]:
c1_m21_experiment.loc['C1_mouse_e10.5_limb_mm10_clean_run3', 'replicates']

['18087_F1',
 '18087_F10',
 '18087_F11',
 '18087_F12',
 '18087_F2',
 '18087_F3',
 '18087_F4',
 '18087_F5',
 '18087_F6',
 '18087_F7',
 '18087_F8',
 '18087_F9',
 '18088_G1',
 '18088_G10',
 '18088_G11',
 '18088_G12',
 '18088_G2',
 '18088_G3',
 '18088_G4',
 '18088_G5',
 '18088_G6',
 '18088_G7',
 '18088_G8',
 '18088_G9',
 '18089_H1',
 '18089_H10',
 '18089_H11',
 '18089_H2',
 '18089_H3',
 '18089_H4',
 '18089_H5',
 '18089_H6',
 '18089_H7',
 '18089_H8',
 '18089_H9']